In [1]:
import pandas as pd

In [2]:
# 곡 정보 파일 불러오기
df = pd.read_csv('../SongTidy/FinalTidy/tidydata/song_tidy03.csv')

In [3]:
# 가수 정보 파일 불러오기
df2 = pd.read_csv('../Data/Data20180921/artist_info_combined_ver04.csv', encoding='euc-kr')

In [4]:
# 곡 정보와 가수 정보를 머지해준다.
song = pd.merge(df, df2, on='artist')

In [5]:
# 가사 분석에 용이하도록 <br>을 공백으로 바꿔준다.
song['lyrics'] = song['lyrics'].str.replace(r'<br>', ' ')

In [6]:
song.shape

(9566, 20)

In [7]:
# 타이틀 곡은 몇곡일까
len(song[song.is_title == '타이틀 곡'])

2379

In [8]:
# !pip install soynlp
# !pip show soynlp

In [9]:
# !pip install gensim
# !pip show gensim

In [10]:
import numpy as np
import re

In [11]:
from soynlp.tokenizer import RegexTokenizer
import gensim
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from gensim import corpora, models
from gensim.models import LdaMulticore
from gensim.models import Word2Vec
from gensim.corpora import Dictionary, MmCorpus


tokenizer = RegexTokenizer()
tokenizer

In [12]:
female = song[song['gender'] == '여']

In [13]:
male = song[song['gender'] == '남']
# 남자아이돌 전체 곡 수
male.shape

(5869, 20)

In [16]:
def preprocessing(text):
    # 개행문자 제거
    text =  text.strip('\t\n\r')
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    # text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [17]:
def tokenize(df):
    sentences = df['lyrics'].apply(preprocessing)
    tokens = sentences.apply(tokenizer.tokenize)
    tokens = tokens.tolist()
    tokens_all = sum(tokens, [])
    return tokens_all

In [18]:
female_t = tokenize(female)
male_t = tokenize(male)
song_t = tokenize(song)

In [19]:
song_t = song_t.copy()
female_t = female_t.copy()
male_t = male_t.copy()

The history saving thread hit an unexpected error (OperationalError('unable to open database file',)).History will not be written to the database.


In [20]:
# 토크나이즈한 단어들의 갯수
len(song_t)

2244770

In [ ]:
import multiprocessing

In [30]:
from collections import Counter

# 정규표현식을 이용하여 어떤 어미가 쓰였는지 알아보자

In [28]:
# 정규표현식을 이용하여 어떤 어미가 쓰였는지 알아보는 함수

def end_of_word(tokenized_list, search_word):
    r = re.compile(search_word)
    result = list(filter(r.match, tokenized_list))
    result = Counter(result).most_common(20)
    return result

In [24]:
# 특정 어미(search_word)의 성별 빈도수 보는 함수(단어 수)

def word_in_song2(search_word):
    r = re.compile(search_word)
    song_r = list(filter(r.match, song_t))
    female_r = list(filter(r.match, female_t))
    male_r = list(filter(r.match, male_t))
    
    song_r = format((len(song_r)/2244770)*1000, '.2f')
    female_r = format((len(female_r)/769211)*1000, '.2f')
    male_r = format((len(male_r)/1409422)*1000, '.2f')
   
    return search_word, song_r, female_r, male_r

In [25]:
# 특정 어미search_word의 성별 빈도수 보는 함수(곡 수)

def word_in_song(search_word):
    song_w = song[song['lyrics'].str.contains(search_word, na=False)]
    female_w = female[female['lyrics'].str.contains(search_word, na=False)]
    male_w = male[male['lyrics'].str.contains(search_word, na=False)]
    
    song_w = format((len(song_w)/9566)*100, '.2f')
    female_w = format((len(female_w)/3338)*100, '.2f')
    male_w = format((len(male_w)/5869)*100, '.2f')
   
    return search_word, song_w, female_w, male_w

In [26]:
r = re.compile(r".주세요$")
result = list(filter(r.match, male_t))

In [27]:
word_in_song2("주세요$")

('주세요$', '0.05', '0.10', '0.02')

In [148]:
word_in_song('beautiful')

('beautiful', '1.89', '1.26', '2.30')

In [115]:
# 어미를 보고 성별 빈도수도 함께 보는 함수
# *단어가 등장한 곡 수* 입니다
# 주의해야 할 점: $가 들어가는 것은 이걸로 함께 볼 수 없다. 위 두 함수로 따로 봐야(성별 빈도수 볼 때는 $ 빼줘야 함. 토크나이즈되지 않았으므로)

def end_of_word2(tokenized_list, search_word):
    r = re.compile(search_word)
    result = list(filter(r.match, tokenized_list))
    result = Counter(result).most_common(20)
    result2 = word_in_song(search_word)
    return result, result2

In [144]:
# 어미를 보고 성별 빈도수도 함께 보는 함수
# *총 단어 빈도수*입니다 곡 수 아님

def end_of_word3(tokenized_list, search_word):
    r = re.compile(search_word)
    result = list(filter(r.match, tokenized_list))
    result = Counter(result).most_common(20)
    result2 = word_in_song2(search_word)
    return result, result2

In [145]:
# 단어 빈도수와 곡 수 같이 보기
# 주의해야 할 점: 역시 $가 들어가는 것은 이걸로 함께 볼 수 없다. word_in_song, end_of_word3 두 함수로 따로 봐야
#(word_in_song(곡 수) 볼 때는 $ 빼줘야 함. 토크나이즈되지 않았으므로)

def end_of_word4(tokenized_list, search_word):
    r = re.compile(search_word)
    result = list(filter(r.match, tokenized_list))
    result = Counter(result).most_common(20)
    result2 = word_in_song(search_word)
    result3 = word_in_song2(search_word)
    return result, result2, result3

### 빈도수는 한 어미의 빈도수를 볼 때 유용합니다.  
r"해(줘|줄|주|요)." 처럼 어떤 어미들이 오는지 볼 때 보다  
r".{1,4}(할|볼|갈|걸|말|뭘|될|갈|놀|알|럴|떨|질|닐|볼|일)까$" 처럼 한 어미를 볼 때  

In [146]:
end_of_word4(female_t, r"해(줘|줄|주|요).")

/Users/yunkim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
/Users/yunkim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/yunkim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  after removing the cwd from sys.path.


([('해줘요', 44),
  ('해줄게', 31),
  ('해주고', 19),
  ('해줄래', 18),
  ('해주세요', 15),
  ('해주는', 10),
  ('해주고픈', 8),
  ('해줄까', 8),
  ('해줄거야', 8),
  ('해주면', 6),
  ('해주오', 6),
  ('해줄래요', 6),
  ('해줄께', 5),
  ('해주길', 5),
  ('해주던', 4),
  ('해주기', 3),
  ('해줘도', 3),
  ('해주지', 3),
  ('해주고만', 3),
  ('해줘야', 2)],
 ('해(줘|줄|주|요).', '27.18', '29.69', '25.63'),
 ('해(줘|줄|주|요).', '0.31', '0.30', '0.31'))

In [98]:
end_of_word(female_t, r"해(줘|줄|주|요).")

[('해줘요', 44),
 ('해줄게', 31),
 ('해주고', 19),
 ('해줄래', 18),
 ('해주세요', 15),
 ('해주는', 10),
 ('해주고픈', 8),
 ('해줄까', 8),
 ('해줄거야', 8),
 ('해주면', 6),
 ('해주오', 6),
 ('해줄래요', 6),
 ('해줄께', 5),
 ('해주길', 5),
 ('해주던', 4),
 ('해주기', 3),
 ('해줘도', 3),
 ('해주지', 3),
 ('해주고만', 3),
 ('해줘야', 2)]

In [97]:
end_of_word(male_t, r"해(줘|줄|주|요).")

[('해줄게', 121),
 ('해주고', 54),
 ('해줄래', 31),
 ('해줘요', 26),
 ('해주는', 22),
 ('해줘서', 17),
 ('해주지', 12),
 ('해주면', 12),
 ('해줄께', 12),
 ('해주길', 7),
 ('해줄거야', 7),
 ('해주겠니', 6),
 ('해줘야만', 6),
 ('해요말고', 6),
 ('해줘야', 5),
 ('해주던', 4),
 ('해주오', 4),
 ('해줄수', 4),
 ('해줄꺼야', 4),
 ('해주세요', 4)]

In [100]:
end_of_word(female_t, r"(돼|되)(줘|줄|어|잖|는|겠|었).")

[('되는걸', 25),
 ('되는데', 22),
 ('돼줄게', 16),
 ('되겠지', 15),
 ('되어준', 13),
 ('되어줘', 12),
 ('되줄게', 12),
 ('되겠니', 10),
 ('되어서', 10),
 ('되었어', 9),
 ('되는거야', 9),
 ('되겠어', 8),
 ('되는지', 8),
 ('되었죠', 8),
 ('돼줄래', 8),
 ('되잖아', 7),
 ('돼줘요', 6),
 ('되어줄', 6),
 ('되어주오', 6),
 ('되어줄게', 5)]

In [101]:
end_of_word(male_t, r"(돼|되)(줘|줄|어|잖|는|겠|었).")

[('되어줘', 53),
 ('되잖아', 51),
 ('되는데', 50),
 ('돼줄게', 41),
 ('되는걸', 39),
 ('되겠어', 37),
 ('되었어', 34),
 ('되겠지', 32),
 ('되어서', 31),
 ('되어줄게', 27),
 ('되겠니', 24),
 ('되는지', 21),
 ('되어준', 19),
 ('되어도', 16),
 ('되었지', 14),
 ('되었죠', 14),
 ('돼줄래', 13),
 ('되어버린', 13),
 ('되는거야', 11),
 ('되었던', 10)]

In [117]:
end_of_word(female_t, r".{1,4}(할|볼|갈|걸|말|뭘|될|갈|놀|알|럴|떨|질|닐|볼|일|하|보|가)(래|자|까)$")

[('아닐까', 46),
 ('말할까', 44),
 ('해볼래', 36),
 ('사랑할래', 29),
 ('멀어질까', 27),
 ('해볼까', 26),
 ('말할래', 25),
 ('보일까', 22),
 ('어떡할래', 22),
 ('이럴까', 21),
 ('가질래', 20),
 ('바라볼래', 17),
 ('안할래', 17),
 ('전할래', 16),
 ('어떨까', 14),
 ('시작할래', 14),
 ('사랑일까', 13),
 ('말해볼까', 12),
 ('다가갈래', 11),
 ('꿈일까', 11)]

In [105]:
end_of_word(male_t, r".{1,4}(할|볼|갈|걸|말|뭘|될|갈|놀|알|럴|떨|질|닐|볼|일|하|보|가)(래|자|까)$")

[('아닐까', 47),
 ('어떨까', 45),
 ('이럴까', 44),
 ('보일까', 35),
 ('말할까', 29),
 ('가보자', 28),
 ('가질래', 23),
 ('해볼까', 22),
 ('사라질까', 22),
 ('말할래', 20),
 ('돌아갈래', 20),
 ('나가자', 18),
 ('사랑일까', 18),
 ('삐뚤어질래', 18),
 ('놀아보자', 17),
 ('왜일까', 16),
 ('잊혀질까', 16),
 ('무엇일까', 15),
 ('걸어가자', 14),
 ('사랑할래', 14)]

In [59]:
end_of_word(female_t, r".{1,4}(할|볼|갈|걸|말|뭘|될|갈|놀|알|럴|떨|질|닐|볼|일)까$")

[('아닐까', 46),
 ('말할까', 44),
 ('멀어질까', 27),
 ('해볼까', 26),
 ('보일까', 22),
 ('이럴까', 21),
 ('어떨까', 14),
 ('사랑일까', 13),
 ('말해볼까', 12),
 ('꿈일까', 11),
 ('걸어볼까', 10),
 ('고백해볼까', 10),
 ('그럴까', 10),
 ('시작할까', 9),
 ('떠나볼까', 8),
 ('다가가볼까', 8),
 ('좋아할까', 8),
 ('왜일까', 8),
 ('착각일까', 7),
 ('시작해볼까', 7)]

In [60]:
end_of_word(male_t, r".{1,4}(할|볼|갈|걸|말|뭘|될|갈|놀|알|럴|떨|질|닐|볼|일)까$")

[('아닐까', 47),
 ('어떨까', 45),
 ('이럴까', 44),
 ('보일까', 35),
 ('말할까', 29),
 ('해볼까', 22),
 ('사라질까', 22),
 ('사랑일까', 18),
 ('왜일까', 16),
 ('잊혀질까', 16),
 ('무엇일까', 15),
 ('욕심일까', 14),
 ('멀어질까', 14),
 ('시작해볼까', 12),
 ('가볼까', 12),
 ('걸어볼까', 11),
 ('전해질까', 11),
 ('변해볼까', 10),
 ('그럴까', 9),
 ('너일까', 9)]

In [62]:
end_of_word(female_t, r".{1,4}(할|볼|갈|말|될|갈|놀|럴|볼)래$")

[('해볼래', 36),
 ('사랑할래', 29),
 ('말할래', 25),
 ('어떡할래', 22),
 ('바라볼래', 17),
 ('안할래', 17),
 ('전할래', 16),
 ('시작할래', 14),
 ('다가갈래', 11),
 ('그만할래', 10),
 ('고백할래', 10),
 ('들어볼래', 8),
 ('놀아볼래', 8),
 ('안아볼래', 7),
 ('유혹할래', 7),
 ('가볼래', 7),
 ('살아갈래', 7),
 ('날아갈래', 6),
 ('즐겨볼래', 6),
 ('뛰어볼래', 6)]

In [63]:
end_of_word(male_t, r".{1,4}(할|볼|갈|말|될|갈|놀|럴|볼)래$")

[('말할래', 20),
 ('돌아갈래', 20),
 ('사랑할래', 14),
 ('바라볼래', 10),
 ('믿어볼래', 9),
 ('다가갈래', 8),
 ('해볼래', 8),
 ('고백할래', 8),
 ('날아갈래', 8),
 ('가볼래', 6),
 ('살아갈래', 6),
 ('그만할래', 6),
 ('걸어볼래', 6),
 ('뭐할래', 6),
 ('데려갈래', 5),
 ('걸어갈래', 5),
 ('지켜볼래', 5),
 ('말해볼래', 5),
 ('노래할래', 5),
 ('떠나볼래', 5)]

In [102]:
end_of_word(female_t, r".{1,3}(하|보|가|놀)자$")

[('그만하자', 7),
 ('달려보자', 6),
 ('살아보자', 6),
 ('노래하자', 4),
 ('뛰어보자', 4),
 ('떠나보자', 4),
 ('나가자', 4),
 ('함께하자', 3),
 ('생각하자', 3),
 ('올라가자', 2),
 ('말해보자', 2),
 ('아파하자', 2),
 ('해보자', 2),
 ('뛰어가자', 2),
 ('어디보자', 2),
 ('두고보자', 2),
 ('설쳐보자', 2),
 ('영원하자', 2),
 ('날아가자', 2),
 ('믿어보자', 2)]

In [103]:
end_of_word(male_t, r".{1,3}(하|보|가|놀)자$")

[('가보자', 28),
 ('나가자', 18),
 ('놀아보자', 17),
 ('걸어가자', 14),
 ('달려보자', 13),
 ('안아보자', 13),
 ('시작하자', 12),
 ('그만하자', 12),
 ('돌아가자', 9),
 ('행복하자', 7),
 ('해보자', 6),
 ('사랑하자', 6),
 ('날아가자', 6),
 ('영원하자', 6),
 ('외워보자', 6),
 ('함께하자', 5),
 ('뛰어보자', 5),
 ('노래하자', 5),
 ('떠나가자', 4),
 ('도전하자', 4)]

In [67]:
end_of_word(female_t, r".{1,4}거야")

[('줄거야', 37),
 ('할거야', 37),
 ('될거야', 35),
 ('않을거야', 28),
 ('갈거야', 25),
 ('없을거야', 22),
 ('온거야', 22),
 ('사랑할거야', 20),
 ('된거야', 19),
 ('하는거야', 18),
 ('올거야', 16),
 ('있을거야', 16),
 ('한거야', 13),
 ('가는거야', 13),
 ('있는거야', 12),
 ('기다릴거야', 12),
 ('없는거야', 11),
 ('말할거야', 11),
 ('되는거야', 9),
 ('말거야', 9)]

In [66]:
end_of_word(male_t, r".{1,4}거야")

[('않을거야', 73),
 ('할거야', 58),
 ('없을거야', 57),
 ('없는거야', 51),
 ('될거야', 39),
 ('있을거야', 37),
 ('있는거야', 36),
 ('된거야', 35),
 ('줄거야', 33),
 ('사랑할거야', 27),
 ('가는거야', 26),
 ('하는거야', 25),
 ('온거야', 25),
 ('말할거야', 23),
 ('말거야', 23),
 ('지켜줄거야', 21),
 ('만들거야', 19),
 ('갈거야', 18),
 ('모를거야', 17),
 ('위한거야', 15)]

In [68]:
end_of_word(female_t, r".{1,4}요$")

[('말아요', 198),
 ('봐요', 190),
 ('필요', 180),
 ('마요', 169),
 ('사랑해요', 150),
 ('있어요', 137),
 ('해요', 129),
 ('와요', 126),
 ('몰라요', 122),
 ('없어요', 114),
 ('있나요', 107),
 ('좋아요', 104),
 ('줘요', 101),
 ('싶어요', 90),
 ('않아요', 90),
 ('안아줘요', 84),
 ('해봐요', 80),
 ('주세요', 79),
 ('없나요', 73),
 ('말해줘요', 71)]

In [73]:
end_of_word(male_t, r".{1,4}요$")

[('필요', 333),
 ('말아요', 327),
 ('마요', 306),
 ('사랑해요', 243),
 ('봐요', 192),
 ('해요', 185),
 ('있어요', 180),
 ('있나요', 174),
 ('없어요', 133),
 ('줘요', 97),
 ('가요', 96),
 ('그래요', 94),
 ('않아요', 87),
 ('알아요', 87),
 ('미안해요', 80),
 ('믿어요', 73),
 ('싶어요', 72),
 ('같아요', 71),
 ('돼요', 66),
 ('여보세요', 63)]

In [75]:
end_of_word(female_t, r".{1,4}줄게$")

[('보여줄게', 95),
 ('해줄게', 31),
 ('지켜줄게', 26),
 ('말해줄게', 22),
 ('안아줄게', 17),
 ('보내줄게', 17),
 ('돼줄게', 16),
 ('비춰줄게', 14),
 ('되줄게', 12),
 ('들어줄게', 12),
 ('들려줄게', 11),
 ('채워줄게', 8),
 ('지워줄게', 7),
 ('잊어줄게', 7),
 ('기다려줄게', 7),
 ('감싸줄게', 6),
 ('잡아줄게', 6),
 ('전해줄게', 6),
 ('되어줄게', 5),
 ('아껴줄게', 5)]

In [74]:
end_of_word(male_t, r".{1,4}줄게$")

[('지켜줄게', 196),
 ('보여줄게', 148),
 ('해줄게', 121),
 ('안아줄게', 71),
 ('돼줄게', 41),
 ('있어줄게', 41),
 ('보내줄게', 30),
 ('되어줄게', 27),
 ('아껴줄게', 22),
 ('비춰줄게', 18),
 ('말해줄게', 17),
 ('지워줄게', 17),
 ('불러줄게', 16),
 ('받아줄게', 15),
 ('잘해줄게', 13),
 ('빌어줄게', 12),
 ('알려줄게', 12),
 ('채워줄게', 12),
 ('감싸줄게', 11),
 ('다해줄게', 11)]

In [95]:
end_of_word(female_t, r".{1,4}줘요$")

[('안아줘요', 84),
 ('말해줘요', 71),
 ('해줘요', 44),
 ('말아줘요', 19),
 ('와줘요', 18),
 ('잡아줘요', 17),
 ('봐줘요', 16),
 ('있어줘요', 15),
 ('보여줘요', 14),
 ('사랑해줘요', 13),
 ('기억해줘요', 11),
 ('웃어줘요', 11),
 ('알아줘요', 9),
 ('지켜줘요', 9),
 ('위로해줘요', 9),
 ('꺼줘요', 9),
 ('속삭여줘요', 7),
 ('구해줘요', 7),
 ('바라봐줘요', 7),
 ('전해줘요', 7)]

In [93]:
end_of_word(male_t, r".{1,4}줘요$")

[('사랑해줘요', 30),
 ('불러줘요', 29),
 ('안아줘요', 28),
 ('해줘요', 26),
 ('있어줘요', 25),
 ('돌아와줘요', 22),
 ('말해줘요', 15),
 ('말아줘요', 13),
 ('기억해줘요', 13),
 ('믿어줘요', 12),
 ('들어줘요', 8),
 ('되어줘요', 7),
 ('알아줘요', 7),
 ('웃어줘요', 7),
 ('받아줘요', 7),
 ('약속해줘요', 7),
 ('용서해줘요', 6),
 ('잡아줘요', 6),
 ('와줘요', 5),
 ('기다려줘요', 5)]

In [86]:
end_of_word(female_t, r".{1,3}세요$")

[('주세요', 79),
 ('해주세요', 15),
 ('안아주세요', 10),
 ('여보세요', 9),
 ('받아주세요', 7),
 ('보세요', 6),
 ('오세요', 6),
 ('놔두세요', 6),
 ('안녕하세요', 6),
 ('하세요', 5),
 ('말해주세요', 5),
 ('찾아오세요', 5),
 ('생각하세요', 4),
 ('지켜내세요', 4),
 ('행복하세요', 4),
 ('느껴보세요', 3),
 ('알려주세요', 3),
 ('비춰주세요', 3),
 ('마세요', 3),
 ('비켜나세요', 2)]

In [84]:
end_of_word(male_t, r".{1,4}세요$")

[('여보세요', 63),
 ('안녕하세요', 46),
 ('마세요', 23),
 ('주세요', 22),
 ('누구세요', 13),
 ('가세요', 13),
 ('가지마세요', 7),
 ('하세요', 7),
 ('행복하세요', 7),
 ('받아주세요', 5),
 ('말해보세요', 5),
 ('일어나세요', 4),
 ('이겨내세요', 4),
 ('느껴보세요', 4),
 ('보세요', 4),
 ('해주세요', 4),
 ('어디세요', 4),
 ('보여주세요', 3),
 ('아껴주세요', 3),
 ('안아주세요', 3)]

In [88]:
end_of_word(female_t, r".{1,3}주세요$")

[('해주세요', 15),
 ('안아주세요', 10),
 ('받아주세요', 7),
 ('말해주세요', 5),
 ('알려주세요', 3),
 ('비춰주세요', 3),
 ('잡아주세요', 2),
 ('알아주세요', 2),
 ('불러주세요', 2),
 ('아껴주세요', 2),
 ('지켜주세요', 2),
 ('만나주세요', 2),
 ('보내주세요', 2),
 ('기억해주세요', 1),
 ('쳐주세요', 1),
 ('위로해주세요', 1),
 ('써주세요', 1),
 ('짜주세요', 1),
 ('도와주세요', 1),
 ('이뤄주세요', 1)]

In [87]:
end_of_word(male_t, r".{1,3}주세요$")

[('받아주세요', 5),
 ('해주세요', 4),
 ('보여주세요', 3),
 ('아껴주세요', 3),
 ('안아주세요', 3),
 ('녹여주세요', 2),
 ('말해주세요', 2),
 ('물어봐주세요', 2),
 ('있어주세요', 2),
 ('용서해주세요', 2),
 ('이해해주세요', 2),
 ('돌려주세요', 2),
 ('대답해주세요', 1),
 ('살아주세요', 1),
 ('잡아주세요', 1),
 ('외처주세요', 1),
 ('춰주세요', 1),
 ('기억해주세요', 1),
 ('불러주세요', 1),
 ('갖다주세요', 1)]

In [89]:
end_of_word(female_t, r".지마$")

[('하지마', 338),
 ('가지마', 100),
 ('보지마', 36),
 ('잊지마', 29),
 ('찾지마', 26),
 ('놓지마', 26),
 ('웃지마', 14),
 ('묻지마', 14),
 ('굴지마', 13),
 ('두지마', 13),
 ('잡지마', 13),
 ('오지마', 13),
 ('울지마', 11),
 ('쓰지마', 9),
 ('팔지마', 8),
 ('듣지마', 8),
 ('빼지마', 7),
 ('속지마', 7),
 ('먹지마', 7),
 ('붙지마', 7)]

In [90]:
end_of_word(male_t, r".지마$")

[('하지마', 522),
 ('가지마', 182),
 ('잊지마', 76),
 ('울지마', 76),
 ('보지마', 56),
 ('찾지마', 29),
 ('놓지마', 27),
 ('막지마', 22),
 ('입지마', 18),
 ('쓰지마', 17),
 ('살지마', 17),
 ('두지마', 16),
 ('묻지마', 13),
 ('있지마', 12),
 ('믿지마', 12),
 ('주지마', 10),
 ('잡지마', 9),
 ('웃지마', 9),
 ('숨지마', 8),
 ('쫄지마', 8)]

In [92]:
end_of_word(female_t, r".{1,4}지마$")

[('하지마', 338),
 ('가지마', 100),
 ('떠나지마', 59),
 ('보지마', 36),
 ('망설이지마', 31),
 ('떠나가지마', 31),
 ('잊지마', 29),
 ('멈추지마', 27),
 ('말하지마', 26),
 ('찾지마', 26),
 ('놓지마', 26),
 ('걱정하지마', 21),
 ('울리지마', 20),
 ('서두르지마', 19),
 ('행복하지마', 17),
 ('바라보지마', 16),
 ('착각하지마', 15),
 ('이러지마', 15),
 ('웃지마', 14),
 ('묻지마', 14)]

In [91]:
end_of_word(male_t, r".{1,4}지마$")

[('하지마', 522),
 ('가지마', 182),
 ('떠나지마', 115),
 ('잊지마', 76),
 ('울지마', 76),
 ('떠나가지마', 73),
 ('걱정하지마', 58),
 ('보지마', 56),
 ('멈추지마', 52),
 ('말하지마', 47),
 ('이러지마', 46),
 ('까불지마', 35),
 ('찾지마', 29),
 ('망설이지마', 27),
 ('생각하지마', 27),
 ('놓지마', 27),
 ('막지마', 22),
 ('버리지마', 20),
 ('입지마', 18),
 ('피하지마', 17)]

In [72]:
# '필요'라는 단어는 어미가 포함되지 않은 단어이므로 제외한다.
import re

r = re.compile("^(?!.*필).*$")
newlist = list(filter(r.match, female_t)) # Read Note
Counter(newlist).most_common(20)

[('내', 9708),
 ('I', 8875),
 ('you', 7537),
 ('난', 6575),
 ('날', 5678),
 ('me', 5513),
 ('oh', 4538),
 ('t', 4415),
 ('나', 4141),
 ('Oh', 4095),
 ('더', 4047),
 ('널', 3990),
 ('love', 3821),
 ('나를', 3647),
 ('it', 3618),
 ('너', 3424),
 ('내가', 3373),
 ('수', 3290),
 ('넌', 3228),
 ('그', 3192)]

In [51]:
unique(before_word(female_t, '해줄게'))  # 여자 아이돌 가사 중 '해줄게' 앞 맥락 보기

[['다시', '믿게', '해줄게'],
 ['곁에만', '있게', '해줄게'],
 ['쿵', '하게', '해줄게'],
 ['이라면', '떠나게', '해줄게'],
 ['너를', '웃게', '해줄게'],
 ['만들어줄게', '그렇게', '해줄게'],
 ['항상', '웃게', '해줄게'],
 ['Alright', '알게', '해줄게'],
 ['대로', '오늘', '해줄게'],
 ['특별히', '뭐든', '해줄게'],
 ['특별히', '허락', '해줄게'],
 ['다시', '느끼게', '해줄게'],
 ['언제까지', '함께', '해줄게'],
 ['오늘처럼만', '웃게', '해줄게'],
 ['후회하지', '않게', '해줄게'],
 ['너를', '깨어나게', '해줄게'],
 ['음악을', '마시게', '해줄게'],
 ['모두', '잊게', '해줄게'],
 ['모르는', '척', '해줄게'],
 ['않아', '부르게', '해줄게'],
 ['다', '잊게', '해줄게']]

In [52]:
unique(before_word(male_t, '해줄게'))  # 남자 아이돌 가사 중 '해줄게' 앞 맥락 보기

[['그대가', '원하는대로', '해줄게'],
 ['언제나', '행복하게', '해줄게'],
 ['내가', '다', '해줄게'],
 ['너를', '웃게', '해줄게'],
 ['준비했던', '말을', '해줄게'],
 ['난', '뭐든', '해줄게'],
 ['매일매일', '웃게', '해줄게'],
 ['끝까지', '함께', '해줄게'],
 ['U', '잘', '해줄게'],
 ['you', '다', '해줄게'],
 ['마', '내가', '해줄게'],
 ['듣고', '느끼게', '해줄게'],
 ['뭐든', '다', '해줄게'],
 ['때로', '간다면', '해줄게'],
 ['꽃길만', '걷게', '해줄게'],
 ['you', '행복하게', '해줄게'],
 ['해줄게', '행복하게', '해줄게'],
 ['해줘', '행복하게', '해줄게'],
 ['너에게', '다', '해줄게'],
 ['위해', '하나도', '해줄게'],
 ['다', '맞출게', '해줄게'],
 ['안', '하면', '해줄게'],
 ['더', '웃게', '해줄게'],
 ['원하면', '다', '해줄게'],
 ['내가', '웃게', '해줄게'],
 ['뭐든지', '다', '해줄게'],
 ['눈감아줄게', '뭐든지', '해줄게'],
 ['어디라도', '함께', '해줄게'],
 ['제대로', '느끼게', '해줄게'],
 ['매일', '느끼게', '해줄게'],
 ['것보다', '더', '해줄게'],
 ['널', '기쁘게', '해줄게'],
 ['마음을', '설레게', '해줄게'],
 ['말자', '행복하게', '해줄게'],
 ['못', '떼게', '해줄게'],
 ['원한다면', '뭐든지', '해줄게'],
 ['널', '웃게', '해줄게'],
 ['보내기엔', '보내기에는', '해줄게'],
 ['아픔', '아물게', '해줄게'],
 ['생각마', '행복하게', '해줄게'],
 ['않게', '잘', '해줄게'],
 ['듯', '매일', '해줄게'],
 ['bye', '뭐든', '해줄게'],
 ['되게', '라도', '해

In [53]:
unique(before_word(female_t, '돼줄게'))

[['막아주고', '빛이', '돼줄게'],
 ['너의', '여자가', '돼줄게'],
 ['밝은', '빛이', '돼줄게'],
 ['네', '편이', '돼줄게'],
 ['너만의', '천사가', '돼줄게'],
 ['걸', '빛이', '돼줄게'],
 ['골목에', '등불이', '돼줄게'],
 ['바다에', '등대가', '돼줄게'],
 ['앙큼한', '고양이', '돼줄게'],
 ['너만의', '빛이', '돼줄게'],
 ['될게', '그늘이', '돼줄게'],
 ['반짝이는', '별이', '돼줄게']]

In [54]:
unique(before_word(male_t, '돼줄게'))

[['사랑의', '포로가', '돼줄게'],
 ['말해', '다', '돼줄게'],
 ['너만의', '수퍼맨이', '돼줄게'],
 ['bae', '북이', '돼줄게'],
 ['이제는', '위로가', '돼줄게'],
 ['style', '내가', '돼줄게'],
 ['그런', '사람', '돼줄게'],
 ['하루', '쉼표가', '돼줄게'],
 ['아파해줄게', '날개가', '돼줄게'],
 ['꽃', '길이', '돼줄게'],
 ['해줄게', '힘이', '돼줄게'],
 ['내가', '힘이', '돼줄게'],
 ['좋은', '사람이', '돼줄게'],
 ['마지막', '빈칸이', '돼줄게'],
 ['되어', '빛이', '돼줄게'],
 ['너의', '침대가', '돼줄게'],
 ['너의', '불씨가', '돼줄게'],
 ['내가', '빛이', '돼줄게'],
 ['너의', '기적이', '돼줄게'],
 ['너의', '편이', '돼줄게'],
 ['아프다면', '그늘이', '돼줄게'],
 ['괜찮은', '너의남자', '돼줄게'],
 ['위해', '집이', '돼줄게'],
 ['너만의', '별이', '돼줄게'],
 ['포근한', '침대가', '돼줄게'],
 ['밤에', '달이', '돼줄게'],
 ['누구보다', '힘이', '돼줄게'],
 ['탑', '스타', '돼줄게']]

In [55]:
unique(before_word(female_t, '될게'))

[['웃는', '거울이', '될게'],
 ['있는', '사람이', '될게'],
 ['속에', '파도가', '될게'],
 ['그대', '힘이', '될게'],
 ['나', '빛이', '될게'],
 ['난', '꽃이', '될게'],
 ['너의', '램프가', '될게'],
 ['내리면', '우산이', '될게'],
 ['될게', '햇살이', '될게']]

In [56]:
unique(before_word(male_t, '될게'))

[['위한', '늑대가', '될게'],
 ['너의', '빛이', '될게'],
 ['위한', '주인공이', '될게'],
 ['그대', '방패가', '될게'],
 ['너의', '이상형', '될게'],
 ['baby', '가', '될게'],
 ['힘들면', '깃털이', '될게'],
 ['그대만의', '꽃길이', '될게'],
 ['뒤로', '내가', '될게'],
 ['위한', '별이', '될게'],
 ['너의', '힘이', '될게'],
 ['맞는', '남자가', '될게'],
 ['멋진', '애인이', '될게'],
 ['내가', '우산이', '될게'],
 ['환한', '빛이', '될게'],
 ['집착하는', '남자', '될게'],
 ['위한', '내가', '될게'],
 ['내가', '빛이', '될게'],
 ['위한', '남자가', '될게'],
 ['밝히는', '빛이', '될게'],
 ['그런', '놈이', '될게'],
 ['내가', '별이', '될게'],
 ['않는', '믿음이', '될게'],
 ['하나뿐인', '믿음이', '될게'],
 ['영원한', '친구가', '될게'],
 ['그런', '남자가', '될게'],
 ['싶은', '나무가', '될게'],
 ['작은', '별이', '될게'],
 ['너의', '하늘이', '될게'],
 ['줄', '한사람이', '될게'],
 ['달려갈', '한사람이', '될게'],
 ['네', '편이', '될게'],
 ['좋은', '남자가', '될게'],
 ['대세는', '내가', '될게'],
 ['baby', '화살이', '될게'],
 ['걸', '배워서는', '될게']]

In [48]:
unique(before_word(female_t, '줘봐'))

[['들이고', '확신을', '줘봐'],
 ['나에게', '미소를', '줘봐'],
 ['너만의', '느낌을', '줘봐'],
 ['사랑을', '보여', '줘봐'],
 ['좀', '해', '줘봐'],
 ['영수증', '좀', '줘봐']]

In [45]:
unique(before_word(female_t, '되어줘'))

[['OH', '슈퍼히어로가', '되어줘'],
 ['OH', '빛이', '되어줘'],
 ['있는', '연인이', '되어줘'],
 ['비밀노래', '열쇠가', '되어줘'],
 ['날', '사랑하게', '되어줘'],
 ['나에게', '추억만', '되어줘'],
 ['나만의', '사랑이', '되어줘'],
 ['나의', '별이', '되어줘'],
 ['친구가', '항상', '되어줘'],
 ['내', '사랑이', '되어줘']]

In [46]:
unique(before_word(male_t, '되어줘'))

[['퍼즐이', '제발', '되어줘'],
 ['저', '달이', '되어줘'],
 ['U', '빛이', '되어줘'],
 ['아름다운', '꿈이', '되어줘'],
 ['나만의', '것', '되어줘'],
 ['어두워', '빛이', '되어줘'],
 ['내게', '우산이', '되어줘'],
 ['내', '짝이', '되어줘'],
 ['밝혀', '빛이', '되어줘'],
 ['못하게', '빛이', '되어줘'],
 ['baby', '빛이', '되어줘'],
 ['oh', '빛이', '되어줘'],
 ['피어나', '빛이', '되어줘'],
 ['e', '빛이', '되어줘'],
 ['없어', '빛이', '되어줘'],
 ['자연스럽게', '리드해', '되어줘'],
 ['나의', '춤이', '되어줘'],
 ['밤하늘에', '빛이', '되어줘'],
 ['빛나는', '별이', '되어줘'],
 ['내게', '빛이', '되어줘'],
 ['영원한', '빛이', '되어줘'],
 ['내', '사랑이', '되어줘'],
 ['마지막', '욕심이', '되어줘'],
 ['밤', '빛이', '되어줘'],
 ['내', '행운이', '되어줘'],
 ['내', '반쪽이', '되어줘'],
 ['내', '여자가', '되어줘'],
 ['ice', 'ice', '되어줘']]

In [265]:
unique(before_word(female_t, '해줘'))

[['사랑', '잊게', '해줘'],
 ['친구처럼', '따뜻하게', '해줘'],
 ['솔직히', '말을', '해줘'],
 ['나를', '생각좀', '해줘'],
 ['생각', '좀', '해줘'],
 ['널', '잊지않게', '해줘'],
 ['널', '떠나게', '해줘'],
 ['사랑했다', '말을', '해줘'],
 ['꼭', '말을', '해줘'],
 ['날', '떠나게', '해줘'],
 ['얘기', '꼭', '해줘'],
 ['웃으면서', '말을', '해줘'],
 ['늘', '함께', '해줘'],
 ['한', '마디만', '해줘'],
 ['언제까지나', '함께', '해줘'],
 ['수', '있게', '해줘'],
 ['제발', '꿈이라', '해줘'],
 ['내게', '말을', '해줘'],
 ['닦고', '행복', '해줘'],
 ['날', '행복하게', '해줘'],
 ['나와', '함께', '해줘'],
 ['나를', '뜨겁게', '해줘'],
 ['나를', '느끼게', '해줘'],
 ['심하게', '말을', '해줘'],
 ['날', '벗어나게', '해줘'],
 ['내게', '고백을', '해줘'],
 ['가슴이', '말을', '해줘'],
 ['잃지', '않게', '해줘'],
 ['안에', '있게', '해줘'],
 ['잊었나', '말을', '해줘'],
 ['진심이', '아니라', '해줘'],
 ['좀더', '말을', '해줘'],
 ['기쁘게', '기쁘게', '해줘'],
 ['느끼게', '느끼게', '해줘'],
 ['뭐든', '말을', '해줘'],
 ['어서', '말을', '해줘'],
 ['살며시', 'kiss', '해줘'],
 ['나를', '설레게', '해줘'],
 ['꺼', '라고', '해줘'],
 ['sorry', '확실히', '해줘'],
 ['No', '확실히', '해줘'],
 ['말', '좀', '해줘'],
 ['난', '모르게', '해줘'],
 ['잊고싶어', '어떻게', '해줘'],
 ['아픔을', '감추게', '해줘'],
 ['나', 'kiss', '해줘']

In [263]:
unique(before_word(male_t, '해줘'))

[['여기서', '쉬게', '해줘'],
 ['없었던', '일로', '해줘'],
 ['내게', '거짓말이라도', '해줘'],
 ['나를', '있게', '해줘'],
 ['나를', '기쁘게만', '해줘'],
 ['결심했다면', '말을', '해줘'],
 ['숨을', '쉴수없게', '해줘'],
 ['어떤', '얘기라도', '해줘'],
 ['안에', '간직', '해줘'],
 ['내게', '전화를', '해줘'],
 ['나란걸', '믿게', '해줘'],
 ['고통받지', '않게', '해줘'],
 ['되도', '모른다', '해줘'],
 ['오히려', '미안하게', '해줘'],
 ['영원히', '함께', '해줘'],
 ['나', '울지않게', '해줘'],
 ['많이', '사랑좀', '해줘'],
 ['사랑', '좀', '해줘'],
 ['한마디만', '내게', '해줘'],
 ['끝을', '벗어나게', '해줘'],
 ['내게', '말을', '해줘'],
 ['얼굴을', '잊게', '해줘'],
 ['내가', '믿게만', '해줘'],
 ['서', '있겠다', '해줘'],
 ['여기서', '같이', '해줘'],
 ['끝나지', '않게', '해줘'],
 ['여기', '서있겠다', '해줘'],
 ['같이', '있게', '해줘'],
 ['나에게', '말을', '해줘'],
 ['세상에', '있기만', '해줘'],
 ['솔직하게', '말을', '해줘'],
 ['보인다고', '말을', '해줘'],
 ['좀', '어떻게', '해줘'],
 ['차라리', '차갑게', '해줘'],
 ['알지', '못하게', '해줘'],
 ['늘', '뿌려놓게', '해줘'],
 ['수', '있게', '해줘'],
 ['보고', '말을', '해줘'],
 ['볼수', '있게', '해줘'],
 ['돌아와', 'Kiss', '해줘'],
 ['더', '말을', '해줘'],
 ['사랑을', '갖게', '해줘'],
 ['깊이', '함께', '해줘'],
 ['한', '번만이라도', '해줘'],
 ['흔들리지', '않게만', '해줘'],
 [

In [34]:
yo = end_of_word(female_t, r".{1,4}요$")

In [35]:
yo

[('말아요', 198),
 ('봐요', 190),
 ('필요', 180),
 ('마요', 169),
 ('사랑해요', 150),
 ('있어요', 137),
 ('해요', 129),
 ('와요', 126),
 ('몰라요', 122),
 ('없어요', 114),
 ('있나요', 107),
 ('좋아요', 104),
 ('줘요', 101),
 ('싶어요', 90),
 ('않아요', 90),
 ('안아줘요', 84),
 ('해봐요', 80),
 ('주세요', 79),
 ('없나요', 73),
 ('말해줘요', 71)]

In [53]:
# 정규표현식을 이용하여 어떤 어미가 쓰였는지 알아보는 함수

r = re.compile(r".{1,4}요$")
result = list(filter(r.match, female_t))

In [54]:
result

['느껴져요',
 '없나요',
 '들어줘요',
 '싶어요',
 '괜찮아요',
 '괜찮아요',
 '느껴져요',
 '들어줘요',
 '싶어요',
 '괜찮아요',
 '괜찮아요',
 '떠나가나요',
 '있었나요',
 '말해줘요',
 '건가요',
 '없나요',
 '줘요',
 '있어줘요',
 '싫어요',
 '말아줘요',
 '있어요',
 '사랑해요',
 '사랑해요',
 '없네요',
 '필요',
 '필요',
 '필요',
 '말아요',
 '모를래요',
 '살래요',
 '아니었나요',
 '사랑해요',
 '헤어져요',
 '들리나요',
 '잊어요',
 '지울까요',
 '닦아요',
 '안되나봐요',
 '사랑해요',
 '헤어져요',
 '들리나요',
 '잊어요',
 '지울까요',
 '미안해요',
 '못해요',
 '원해요',
 '없어요',
 '중요',
 '있어요',
 '없어요',
 '있네요',
 '슬퍼져요',
 '없어요',
 '원하잖아요',
 '느끼나요',
 '있어요',
 '없어요',
 '있어요',
 '없어요',
 '없어요',
 '다가갈래요',
 '얘기할래요',
 '다가갈래요',
 '얘기할래요',
 '가나요',
 '필요해요',
 '안아줘요',
 '안아줘요',
 '안아줘요',
 '안아줘요',
 '안아줘요',
 '들리나요',
 '들리나요',
 '다가와줘요',
 '줘요',
 '줘요',
 '께요',
 '잊지마요',
 '안아줘요',
 '잊지마요',
 '안아줘요',
 '잊지마요',
 '안아줘요',
 '고백해요',
 '고백해요',
 '기회예요',
 '고백해요',
 '말을해요',
 '고백해줘요',
 '고백해요',
 '고백해요',
 '기회예요',
 '돼요',
 '고백해요',
 '고백해요',
 '기회예요',
 '빛나요',
 '안아줄래요',
 '말해요',
 '말해요',
 '줄께요',
 '싶어요',
 '할께요',
 '들어요',
 '참아봐요',
 '기다려요',
 '들려요',
 '줘요',
 '이예요',
 '다가와요',
 '다가와요',
 '싶어요',
 '말해요',
 '말해요',
 '줄께요',
 '굴지마요',
 '마요

In [55]:
# '필요'라는 단어는 어미가 포함되지 않은 단어이므로 제외한다.
# import re

# text = re.sub(pattern, ' ', text)

result2 = re.sub("^(?!.*필).*$", '', str(result))
result2

"['느껴져요', '없나요', '들어줘요', '싶어요', '괜찮아요', '괜찮아요', '느껴져요', '들어줘요', '싶어요', '괜찮아요', '괜찮아요', '떠나가나요', '있었나요', '말해줘요', '건가요', '없나요', '줘요', '있어줘요', '싫어요', '말아줘요', '있어요', '사랑해요', '사랑해요', '없네요', '필요', '필요', '필요', '말아요', '모를래요', '살래요', '아니었나요', '사랑해요', '헤어져요', '들리나요', '잊어요', '지울까요', '닦아요', '안되나봐요', '사랑해요', '헤어져요', '들리나요', '잊어요', '지울까요', '미안해요', '못해요', '원해요', '없어요', '중요', '있어요', '없어요', '있네요', '슬퍼져요', '없어요', '원하잖아요', '느끼나요', '있어요', '없어요', '있어요', '없어요', '없어요', '다가갈래요', '얘기할래요', '다가갈래요', '얘기할래요', '가나요', '필요해요', '안아줘요', '안아줘요', '안아줘요', '안아줘요', '안아줘요', '들리나요', '들리나요', '다가와줘요', '줘요', '줘요', '께요', '잊지마요', '안아줘요', '잊지마요', '안아줘요', '잊지마요', '안아줘요', '고백해요', '고백해요', '기회예요', '고백해요', '말을해요', '고백해줘요', '고백해요', '고백해요', '기회예요', '돼요', '고백해요', '고백해요', '기회예요', '빛나요', '안아줄래요', '말해요', '말해요', '줄께요', '싶어요', '할께요', '들어요', '참아봐요', '기다려요', '들려요', '줘요', '이예요', '다가와요', '다가와요', '싶어요', '말해요', '말해요', '줄께요', '굴지마요', '마요', '다가와요', '몰라요', '놀라요', '건가요', '쓰러져요', '해요', '말해줘요', '말해줘요', '말해줘요', '떠나요', '말해줘요', '말해줘요', '말해줘요', '말해줘요', '말해줘요'

In [56]:
Counter(result).most_common(20)

[("'", 17046),
 ('요', 8550),
 (',', 8522),
 (' ', 8522),
 ('아', 1317),
 ('해', 1184),
 ('어', 1166),
 ('나', 967),
 ('줘', 685),
 ('봐', 507),
 ('가', 438),
 ('말', 430),
 ('있', 379),
 ('네', 301),
 ('래', 288),
 ('마', 267),
 ('려', 260),
 ('없', 250),
 ('게', 247),
 ('안', 245)]

In [33]:
re.sub?

In [ ]:
word_in_song2(female_t, r".{1,4}요$")

In [ ]:
# tf-idf로 여자 아이돌의 가사에만 많이 나오는 단어, 남자 아이돌의 가사에만 많이 나오는 단어를 보고 싶다.